 - Notebook for training of various convolutional regression models

In [1]:
# # if running on a GPU machine, uncomment this cell and select a GPU by changing the CUDA_VISIBLE_DEVICES index
# # in terminal to view active GPUs:
# # watch -d -n 0.5 nvidia-smi

# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../modules/'))
sys.path.insert(1, os.path.join(sys.path[0], '../data_creation/'))
sys.path.insert(1, os.path.join(sys.path[0], '../networks/'))
sys.path.insert(1, os.path.join(sys.path[0], '../network_trainers/'))

from CNN_R_Trainer import CNN_R_Trainer
from CNN_DS_BN_R_Trainer import CNN_DS_BN_R_Trainer
from Data_Creator_R import Data_Creator_R as Data_Creator
from CNN_QP_BN_R import CNN_QP_BN_R
from CNN_DS_BN_R import CNN_DS_BN_R

from data_manipulation import *

## Load & Setup Data

In [3]:
# extract the redundant baselines and their gains and data from miriad and calfits files
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

In [3]:
# seperate trining and testing redundant baselines 
# if we have not already done this, load them from disk
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())

# seperate the visiblites
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

In [18]:
CNN_DS_BN_R_H = CNN_DS_BN_R('CNN_DS_BN_R_H', 7, cost = 'MQE', adam_initial_learning_rate=1e-6)

In [19]:
CNN_DS_BN_R_H.print_params()

{'accuracy_threshold': 0.00625,
 'adam_initial_learning_rate': 1e-06,
 'cost': 'MQE',
 'gaussian_shift_scalar': 1e-05,
 'log_dir': 'logs/',
 'name': 'CNN_DS_BN_R_H',
 'num_downsamples': 7}


In [22]:
trainer = CNN_DS_BN_R_Trainer(CNN_DS_BN_R_H, Data_Creator, batch_size=2, num_epochs=200)

In [23]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains,
                 num_flatnesses = 2)

In [24]:
trainer.print_params()

{'batch_size': 2,
 'downsample_keep_prob': 0.9,
 'log_dir': 'logs/',
 'metric_names': ['MISGs', 'MSEs', 'MQEs', 'PWTs'],
 'model_save_interval': 25,
 'name': 'CNN_DS_BN_R_H',
 'num_epochs': 200,
 'pretrained_model_path': None,
 'sample_keep_prob': 0.8}


In [25]:
trainer.train()

saving metrics - saved